<a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%206%20-%20Vertex%20AI/vertex_ai_raw.ipynb" target="_blank">
  <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>

# Install Additional Packages
First off, you'll also need to install a few packages.

In [ ]:
!pip install --quiet google-cloud-storage
!pip install --quiet google.cloud.aiplatform

# Restart the Kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.  When you run this, you may get a notification that the kernel crashed.  You can disregard that.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Download and Split the Data
Now let's download the data set and split it into training, validation and test sets.

In [ ]:
!wget https://storage.googleapis.com/neo4j-datasets/form13/2021.csv

In [ ]:
import pandas
df = pandas.read_csv('2021.csv')

train = df.loc[df['reportCalendarOrQuarter'] == '03-31-2021']
validate = df.loc[df['reportCalendarOrQuarter'] == '06-30-2021']
test = df.loc[df['reportCalendarOrQuarter'] == '09-30-2021']

df.to_csv('raw.csv', index=False)

# Authenticate your Google Cloud Account
These steps will authenticate the notebook using your Google Cloud credentials.

In [ ]:
# Edit these variables!
PROJECT_ID = 'YOUR-PROJECT-ID'
STORAGE_BUCKET = 'NAME-OF-BUCKET-FROM-PREVIOUS-LAB'

# You can leave these defaults
REGION = 'us-central1'

In [ ]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [ ]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Upload to a GCP Cloud Storage Bucket

To get the data into Vertex AI, we must first put it in a bucket as a CSV.

In [ ]:
from google.cloud import storage
client = storage.Client()

In [ ]:
bucket = client.bucket(STORAGE_BUCKET)

In [ ]:
filename = 'raw.csv'
upload_path = os.path.join('form13', filename)
blob = bucket.blob(upload_path)
blob.upload_from_filename(filename)

# Train a Model on GCP
We'll use the original features to train an AutoML model.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="form13-raw",
    gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'form13', 'raw.csv'),
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='form13-raw',
    optimization_prediction_type='classification'
)

In [ ]:
model = job.run(
    dataset=dataset,
    target_column='target',
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name='form13-raw',
    disable_early_stopping=False,
    budget_milli_node_hours=1000,
)

This job will run for an hour.  That's the minimum time for an AutoML job.  We're going to move on while that runs.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.